In [0]:
from preprocessing import *
from clustering import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from GMM import my_GMM_uniform_cluster
data_contrats = pd.read_csv('train_contrats_approx.csv',sep=",")
from sklearn.metrics import adjusted_rand_score
from collections import Counter

Le but est ici d'évaluer la stabilité de differentes méthodes de clustering, en utilisant le procédé suivant : 
Clustering sur un jeu d'entrainement (75%) (Clustering A)
Nouveau clustering sur toute la base de données, ie 75% initaux + 25% restants (Clustering B)
Calcul du adjusted rand index, qui est d'autant plus grand que deux points dans un même cluster dan le clustering A sont toutjours dans le même cluster dans le clustering B, ou que deux points dans deux clusters différents dans le clustering A sont toutjours dans deux clusters différents dans le clustering B.
Lors du calcul du score, seuls 75% des points sont donc considérés.
Selon nous, une perturbation de 25% est forte, et permettera de vraiment mettre les méthodes de clustering à l'épreuve.

In [0]:
data = pd.read_csv("train_contrats_approx.csv", sep=",", engine="python")

# For original database ---
#data = pd.read_csv("train_contrats.csv", sep=";")
#data["Freq_sinistre"] = data["nombre_de_sinistre"]/data["Exposition_au_risque"]

#train_data, _ = preprocessing(data, balance=False, train_size=0.75)
preprocessing(data, balance=False, train_size=1.)[0]
data = preprocessing(data, balance=False, train_size=1.)[0]
train_data = data[:(int(len(data)*0.75))]

In [3]:
train_data

,Classe_Age_Situ_Cont,franchise,Exposition_au_risque,ValeurPuissance,Freq_sinistre,Type_Apporteur_1,Type_Apporteur_2,Type_Apporteur_3,Activite_1,Activite_2,Activite_3,Activite_4,Activite_5,Activite_6,Activite_7,Activite_8,Mode_gestion_N,Mode_gestion_P,Zone_1,Zone_2,Zone_3,Zone_4,Zone_5,Zone_6,Fractionnement_A,Fractionnement_S,Fractionnement_T,FORMULE_2,FORMULE_3,FORMULE_4,FORMULE_5,Age_du_vehicule_<= 5 ans,Age_du_vehicule_> 5 ans
0,0.596130,0.298065,0.054645,0.695485,0.000183,0.099355,0.000000,0.0,0.0,0.0,0.099355,0.0,0.0,0.0,0.000000,0.0,0.099355,0.000000,0.000000,0.0,0.0,0.099355,0.0,0.0,0.099355,0.000000,0.000000,0.0,0.000000,0.0,0.099355,0.099355,0.000000
1,0.541791,0.216716,0.044427,0.758507,0.000274,0.000000,0.108358,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.108358,0.0,0.108358,0.000000,0.000000,0.0,0.0,0.108358,0.0,0.0,0.000000,0.108358,0.000000,0.0,0.108358,0.0,0.000000,0.000000,0.108358
2,0.542157,0.216863,0.024939,0.759020,0.000262,0.000000,0.108431,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.108431,0.0,0.108431,0.000000,0.000000,0.0,0.0,0.108431,0.0,0.0,0.000000,0.108431,0.000000,0.0,0.108431,0.0,0.000000,0.000000,0.108431
3,0.692925,0.277170,0.036032,0.554340,0.000773,0.000000,0.138585,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.138585,0.0,0.138585,0.000000,0.000000,0.0,0.0,0.138585,0.0,0.0,0.000000,0.138585,0.000000,0.0,0.000000,0.0,0.138585,0.138585,0.000000
4,0.755575,0.251858,0.030223,0.503717,0.004656,0.000000,0.125929,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.125929,0.0,0.125929,0.000000,0.000000,0.0,0.0,0.125929,0.0,0.0,0.000000,0.125929,0.000000,0.0,0.000000,0.0,0.125929,0.125929,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65416,0.110427,0.331280,0.110427,0.883414,0.009290,0.110427,0.000000,0.0,0.0,0.0,0.110427,0.0,0.0,0.0,0.000000,0.0,0.000000,0.110427,0.110427,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.110427,0.0,0.110427,0.0,0.000000,0.110427,0.000000
65417,0.110510,0.331530,0.103879,0.884080,0.000000,0.110510,0.000000,0.0,0.0,0.0,0.110510,0.0,0.0,0.0,0.000000,0.0,0.000000,0.110510,0.110510,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.110510,0.0,0.110510,0.0,0.000000,0.110510,0.000000
65418,0.216923,0.325384,0.108461,0.867690,0.008504,0.108461,0.000000,0.0,0.0,0.0,0.108461,0.0,0.0,0.0,0.000000,0.0,0.000000,0.108461,0.108461,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.108461,0.0,0.108461,0.0,0.000000,0.000000,0.108461
65419,0.239046,0.358569,0.119523,0.836660,0.000000,0.119523,0.000000,0.0,0.0,0.0,0.119523,0.0,0.0,0.0,0.000000,0.0,0.000000,0.119523,0.119523,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.119523,0.0,0.119523,0.0,0.000000,0.119523,0.000000


#### Adjusted rand index

Pour avoir une baseline et mieux intepreter l'ARI, il est calculé ici pour un clustering **aléatoire**, pour un nombre de point à peu près égal à celui de notre base de données. L'ARI devrait dans ce cas valoir 0.

In [0]:
ari = 0
for i in range(100):
  clust_A = np.random.randint(low=0, high=4, size=65000)
  clust_B = np.random.randint(low=0, high=4, size=65000)
  ari += adjusted_rand_score(clust_A,clust_B)/100

In [30]:
print(ari)

8.920638121083689e-07


### KMeans

Essayons avec 5 clusters pour commencer !

Train data

In [0]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(train_data.drop(["Freq_sinistre"],axis=1))
train_labels=kmeans.labels_

In [36]:
train_labels

array([2, 2, 2, ..., 1, 1, 1], dtype=int32)

All data

In [0]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(data.drop(["Freq_sinistre"],axis=1))
labels=kmeans.labels_

In [0]:
labels = labels[:len(train_labels)]

In [39]:
adjusted_rand_score(labels,train_labels)

0.6554226428562993

Le score ARI est nettement meilleur qu'avec les autres méthodes.

In [41]:
for n_clusters in range(1,10):
  kmeans_train = KMeans(n_clusters=n_clusters, random_state=0).fit(train_data.drop(["Freq_sinistre"],axis=1))
  train_labels=kmeans_train.labels_

  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(data.drop(["Freq_sinistre"],axis=1))
  labels=kmeans.labels_

  labels = labels[:len(train_labels)]
  ari = adjusted_rand_score(labels,train_labels)
  print(f"{n_clusters} clusters : ARI = {ari}")


1 clusters : ARI = 1.0
2 clusters : ARI = 0.42320404919743526
3 clusters : ARI = 0.6989080841732364
4 clusters : ARI = 0.6509929162002065
5 clusters : ARI = 0.6554226428562993
6 clusters : ARI = 0.48580210153293424
7 clusters : ARI = 0.447221279087362
8 clusters : ARI = 0.6150717139609658
9 clusters : ARI = 0.5631164380055969
